# Lecture 4

### ULMfit for fine tuning (RNNs)
- Wikitext103 : language model for next word predication (training from scracth)
- IMDB : language model (took pretrained model from wikipedia model)
- IMDB : sentiment classifier for reviews (fine tuned the language model of IMDB)


### US Patent Phase to Phase Competition
- use "include = object" in dataframe describe function for text columns
- Hugging face model hub : pretrained models (specific to tasks as well)
- Transformers : bit challenging with long documents as it processes in one go (documnets < 2000 words else need GPUs with large memory)
- ULMfit : process gradually by dividing into pieces (documnets > 2000 words)
- Data split : training, validation and test
- outliers : no need to remove all the time

### Transformer library from hugging face
- using classification model and passing continuous variable as lable : does it detect automatically?